# Data preprocessing

In [1]:
import numpy as np              # Одномерные и многомерные массивы (array)
import pandas as pd             # Таблицы и временные ряды (dataframe, series)
import matplotlib.pyplot as plt # Научная графика
# import seaborn as sns           # Еще больше красивой графики для визуализации данных
import sklearn                  # Алгоритмы машинного обучения

In [2]:
data_raw = pd.read_csv('SBAnational.csv', low_memory=False)

In [3]:
data_raw.shape

(899164, 27)

In [4]:
data_raw.head(5)

,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,...,RevLineCr,LowDoc,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,451120,28-Feb-97,1997,...,N,Y,NaN,28-Feb-99,"$60,000.00",$0.00,P I F,$0.00,"$60,000.00","$48,000.00"
1,1000024006,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,46526,1ST SOURCE BANK,IN,722410,28-Feb-97,1997,...,N,Y,NaN,31-May-97,"$40,000.00",$0.00,P I F,$0.00,"$40,000.00","$32,000.00"
2,1000034009,"WHITLOCK DDS, TODD M.",BLOOMINGTON,IN,47401,GRANT COUNTY STATE BANK,IN,621210,28-Feb-97,1997,...,N,N,NaN,31-Dec-97,"$287,000.00",$0.00,P I F,$0.00,"$287,000.00","$215,250.00"
3,1000044001,"BIG BUCKS PAWN & JEWELRY, LLC",BROKEN ARROW,OK,74012,1ST NATL BK & TR CO OF BROKEN,OK,0,28-Feb-97,1997,...,N,Y,NaN,30-Jun-97,"$35,000.00",$0.00,P I F,$0.00,"$35,000.00","$28,000.00"
4,1000054004,"ANASTASIA CONFECTIONS, INC.",ORLANDO,FL,32801,FLORIDA BUS. DEVEL CORP,FL,0,28-Feb-97,1997,...,N,N,NaN,14-May-97,"$229,000.00",$0.00,P I F,$0.00,"$229,000.00","$229,000.00"


In [5]:
data_raw = data_raw.drop(columns=['Name', 'Zip', 'LowDoc', 'RevLineCr'])

In [6]:
def convert_from_str_to_float(column):
    column = column.str.removeprefix("$")
    column = column.str.removesuffix(" ")
    column = column.str.replace(',', '')
    column = column.astype('float64')

    return column

In [7]:
data_raw['City'] = data_raw['City'].astype('category')
data_raw['State'] = data_raw['State'].astype('category')
data_raw['Bank'] = data_raw['Bank'].astype('category')
data_raw['BankState'] = data_raw['BankState'].astype('category')
data_raw['MIS_Status'] = data_raw['MIS_Status'].astype('category')

data_raw['DisbursementGross'] = convert_from_str_to_float(data_raw['DisbursementGross'])
data_raw['BalanceGross'] = convert_from_str_to_float(data_raw['BalanceGross'])
data_raw['ChgOffPrinGr'] = convert_from_str_to_float(data_raw['ChgOffPrinGr'])
data_raw['GrAppv'] = convert_from_str_to_float(data_raw['GrAppv'])
data_raw['SBA_Appv'] = convert_from_str_to_float(data_raw['SBA_Appv'])

In [8]:
data_raw['NAICS'] = data_raw['NAICS'] // 10000
data_raw['NAICS'].astype('category')

data_raw = data_raw[data_raw['MIS_Status'].notna()]

In [9]:
data_raw['ApprovalDate'] = pd.to_datetime(data_raw['ApprovalDate'])

def func(elem):
    return elem.year

data_raw['ApprovalDate'].apply(func)

# for ind, elem in enumerate(data_raw['ApprovalDate']):
#     data_raw['ApprovalDate'] = elem.year
# data_raw['ApprovalDate'].value_counts()

# data_raw[data_raw['ApprovalFY'] == '1976A'] = 1976
# data_raw['ApprovalFY'] =  data_raw['ApprovalFY'].astype('int64')

/tmp/ipykernel_5605/3938579074.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data_raw['ApprovalDate'] = pd.to_datetime(data_raw['ApprovalDate'])


0         1997
1         1997
2         1997
3         1997
4         1997
          ... 
899159    1997
899160    1997
899161    1997
899162    1997
899163    1997
Name: ApprovalDate, Length: 897167, dtype: int64

In [10]:
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 897167 entries, 0 to 899163
Data columns (total 23 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   LoanNr_ChkDgt      897167 non-null  int64         
 1   City               897137 non-null  category      
 2   State              897154 non-null  category      
 3   Bank               895661 non-null  category      
 4   BankState          895654 non-null  category      
 5   NAICS              897167 non-null  int64         
 6   ApprovalDate       897167 non-null  datetime64[ns]
 7   ApprovalFY         897167 non-null  object        
 8   Term               897167 non-null  int64         
 9   NoEmp              897167 non-null  int64         
 10  NewExist           897033 non-null  float64       
 11  CreateJob          897167 non-null  int64         
 12  RetainedJob        897167 non-null  int64         
 13  FranchiseCode      897167 non-null  int64        

In [11]:
data_raw.describe()

,LoanNr_ChkDgt,NAICS,ApprovalDate,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,DisbursementGross,BalanceGross,ChgOffPrinGr,GrAppv,SBA_Appv
count,8.971670e+05,897167.000000,897167,897167.000000,897167.000000,897033.000000,897167.000000,897167.000000,897167.000000,897167.000000,8.971670e+05,897167.000000,8.971670e+05,8.971670e+05,8.971670e+05
mean,4.774982e+09,39.600395,2001-05-31 18:17:52.504003712,110.848592,11.412562,1.280403,8.444305,10.807308,2757.528287,0.757274,2.015980e+05,2.996003,1.352721e+04,1.930595e+05,1.497807e+05
min,1.000014e+09,0.000000,1973-02-06 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000e+03,0.000000,0.000000e+00,1.000000e+03,5.000000e+02
25%,2.593070e+09,23.000000,1996-11-20 00:00:00,60.000000,2.000000,1.000000,0.000000,0.000000,1.000000,0.000000,4.249200e+04,0.000000,0.000000e+00,3.500000e+04,2.125000e+04
50%,4.363894e+09,44.000000,2002-09-26 00:00:00,84.000000,4.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000e+05,0.000000,0.000000e+00,9.000000e+04,6.205000e+04
75%,6.908644e+09,56.000000,2006-01-20 00:00:00,120.000000,10.000000,2.000000,1.000000,4.000000,1.000000,1.000000,2.390000e+05,0.000000,0.000000e+00,2.250000e+05,1.750000e+05
max,9.996003e+09,92.000000,2072-12-08 00:00:00,569.000000,9999.000000,2.000000,8800.000000,9500.000000,99999.000000,2.000000,1.144632e+07,996262.000000,3.512596e+06,5.472000e+06,5.472000e+06
std,2.537370e+09,26.288765,NaN,78.900862,73.793775,0.451741,236.950249,237.382398,12767.025962,0.646540,2.878066e+05,1443.766066,6.520986e+04,2.834331e+05,2.285600e+05


In [12]:
data_raw.describe(include=['category'])

,City,State,Bank,BankState,MIS_Status
count,897137,897154,895661,895654,897167
unique,32566,51,5801,56,2
top,LOS ANGELES,CA,BANK OF AMERICA NATL ASSOC,CA,P I F
freq,11550,130488,86773,118034,739609


In [13]:
data_raw.isnull().sum()

LoanNr_ChkDgt             0
City                     30
State                    13
Bank                   1506
BankState              1513
NAICS                     0
ApprovalDate              0
ApprovalFY                0
Term                      0
NoEmp                     0
NewExist                134
CreateJob                 0
RetainedJob               0
FranchiseCode             0
UrbanRural                0
ChgOffDate           734729
DisbursementDate       2175
DisbursementGross         0
BalanceGross              0
MIS_Status                0
ChgOffPrinGr              0
GrAppv                    0
SBA_Appv                  0
dtype: int64

# Data analyze

In [14]:
from pandas.plotting import scatter_matrix
scatter_matrix(data_raw, alpha = .01, figsize = (30, 30))
pass

# data_raw.plot(kind='pie', x='NAICS', y='MIS_Status', alpha = .01)
# pass